In [1]:
import re
import random
import os
import numpy as np
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import logging
from sklearn.preprocessing import LabelBinarizer


class JustLSTM(keras.Model):
    def __init__(self, input_shape, timeframe, LSTM_units, dense_units, num_classes=4):
        super().__init__()

        # TODO: kijken of we input shape niet uit elkaar trekken in width, height, batch_size en timesteps (nu num_segments)
        self.in_shape = input_shape
        self.num_classes = num_classes
        self.LSTM_units = LSTM_units

        # The number of images / timesteps that we will look at for each training step
        self.timeframe = timeframe

        self.lstm1 = keras.layers.LSTM(self.LSTM_units, return_sequences=True, name="lstm1")
        self.lstm2 = keras.layers.LSTM(self.LSTM_units, name="lstm2")

        self.fc = keras.layers.Dense(dense_units, activation="relu")

        self.out = keras.layers.Dense(self.num_classes, activation="softmax", name="output")

    def call(self, inputs, training=True, mask=None):
        """
        Specifies how the inputs should be passed through the layers of the model and returns the output

        :param inputs: the inputs to be classified by the model, with the same shape as self.in_shape
        :param training: whether the gradients should be tracked
        :param mask: whether a certain mask should be applied on the inputs (such as masking certain timesteps)
        :return: probabilities for each of the classes
        """

        input_layer = keras.layers.InputLayer(self.in_shape, name="input")(inputs)

        # Pass the timesteps through the RNN to find temporal features The amount of units in the layer are equal to
        # the number of timesteps (i.e. segments) according to Zhang et al. (2018)
        lstm1 = self.lstm1(input_layer)
        lstm2 = self.lstm2(lstm1)

        #fc = self.fc(lstm2)

        # Final fully connected layer with softmax to give class probabilities
        output = self.out(lstm2)

        return output

    def build_graph(self):
        """
        Builds the Tensor Graph in order to generate a summary of the model by running dummy input through the model
        :return: a 'dummy' version of the model of which we can generate a summary
        """
        x = keras.Input(self.in_shape)
        return keras.Model(inputs=[x], outputs=self.call(x))

In [3]:
def create_windows(dataset, model_type, timeframe):
    # Will contain the slices of the dataset that represent the different windows
    windows = []

    if model_type == "cascade":
        num_windows = int(dataset.shape[2] / timeframe)
    else:
        num_windows = int(dataset.shape[1] / timeframe)

    i = timeframe
    j = 0
    count = 0

    # Loop through the dataset until we have the specified number of windows (might cut off some of the last timesteps)
    while count < num_windows:
        # Save the view of the array in the windows list
        if model_type == "cascade":
            view = dataset[:, :, j:i]
        else:
            view = dataset[:, j:i]
        windows.append(view)

        i += timeframe
        j += timeframe
        count += 1
    # Return the list of array views
    return windows


def read_prepro_file(data_path):
    hfive = h5py.File(data_path, 'r')
    matrix = hfive.get('dir')
    matrix = np.array(matrix)
    return matrix

In [4]:
import tensorflow as tf
from tensorflow import keras
import re
import os
import random
import numpy as np


# tf function zal niet heel veel uitmaken volgens de documentatie, omdat wij veel convolutional operations hebben (waarbij de speedup dus meevalt)
def train_epoch(model, x_train, y_train, optimizer, train_acc):
    # Instantiate an optimizer and loss function
    loss_function = keras.losses.CategoricalCrossentropy()

    total_loss = 0

    # Doet nu nog wel een update per window, wellicht veel? We kunnen ook nog batches gaan gebruiken om het aantal updates te reduceren
    for step in range(len(x_train)):
        # Used to track the gradients during the forward pass
        with tf.GradientTape() as tape:
            logits = model(tf.expand_dims(x_train[step], axis=0), training=True)
            # Calculate the loss value
            loss_value = loss_function(y_train[step], logits)

        total_loss += loss_value

        # Extract the gradients from the gradienttape
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Perform a weight update step using the extracted gradients
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        # Keep track of the training accuracy which is given at the end of the loop
        train_acc.update_state(y_train[step], logits)

        # Report the training loss for monitoring
        if step % 100 == 0:
            print("Training loss value at step {}: {}".format(step, loss_value))

    print("Training accuracy over whole file: {}".format(train_acc.result()))
    print("---------------------------------------------------")
    average_loss = total_loss / len(x_train)
    return average_loss, train_acc


# TODO: kijken of we de input als tensors willen doen (wat wel netter is I guess) of makkelijker als np arrays --> netwerk vindt allebei goed
def fit_eight(model, model_type, dirlist, parent_path, optimizer, timeframe, label_encoder,
              train_acc_obj):  # Fit a network on 8 datafiles at a time.
    fit_list = []
    average_loss = 0
    i = 0
    j = 0
    while i < 8:  # Load the 8 datafiles
        filename = parent_path + "/" + dirlist[i]

        data = read_prepro_file(filename)
        label = extract_label(filename)

        fit_list.append((data, label))
        i += 1

    for counter, (data, label) in enumerate(fit_list):
        print("Fitting on file {} containing task: {}".format((counter + 1), label))
        # Creates windows of length 'timeframe' to fit the model on
        windows = create_windows(data, model_type, timeframe)
        x_train = []
        # Only needed for CascadeNet, not for the LSTM network
        for window in windows:
            if model_type == "cascade":
                x_train.append(tf.convert_to_tensor(window))
            else:
                x_train.append(tf.convert_to_tensor(np.transpose(window)))

        # Encode the textual label to one-hot-encoding
        encoded_label = label_encoder.transform([label])
        # Create a list of tensors, each corresponding to an encoded label
        y_train = [tf.convert_to_tensor(encoded_label)] * len(x_train)

        # Fit the model on the data from this specific file
        average_loss, train_acc_obj = train_epoch(model, x_train, y_train, optimizer, train_acc_obj)

    return average_loss, train_acc_obj


def train_dir(model, model_type, dirpath, epochs, timeframe, optimizer, label_encoder,
              shuffle=True):  # Train a network on a given directory
    dirnames = os.listdir(dirpath)  # get list of all filenames
    random.shuffle(dirnames)
    batches = len(dirnames) // 8  # int divide by eight to avoid float errors

    train_acc_obj = keras.metrics.CategoricalAccuracy()

    average_epoch_accuracy = 0
    average_epoch_loss = 0
    losses = []
    accuracies = []

    # Check if it is divisible by eight
    if not (len(dirnames) % 8) == 0:
        raise TypeError("Not divisible by eight")

    for epoch in range(epochs):
        print("Starting epoch {}".format(epoch + 1))
        print()
        train_acc_obj.reset_states()
        for i in range(batches):  # Fit on 8 batches at a time                HIER IPV 1 WEER 'BATCHES' NEERZETTEN
            print("Fitting on files {}-{}....".format((i * 8), ((i + 1) * 8)))
            # Fits the model on 8 files for the specified amount of epochs
            average_loss, train_acc_obj = fit_eight(model, model_type, dirnames[i * 8:(i + 1) * 8], dirpath, optimizer,
                                                    timeframe, label_encoder, train_acc_obj)
            average_epoch_loss += average_loss
            average_epoch_accuracy += train_acc_obj.result()

            print("--------------------------------------------------------------")

        losses.append(average_epoch_loss)
        accuracies.append(average_epoch_accuracy)

    # SAVE THE MODEL AFTER TRAINING!!!!
    model.save("models/trained_LSTM_e2_5_20_intra")

    return model, losses, accuracies


def extract_label(filename):  # Extract the label out of a filename
    filename = filename.split("/")
    pattern = r'_\d'
    split = re.split(pattern, filename[-1])
    return split[0]


In [5]:
from sklearn.preprocessing import LabelBinarizer
import tensorflow
from tensorflow import keras

output_classes = 4

# Hyperparameters
timeframe = 5
LSTM_units = 20
# (1e-2, 1e-3, 1e-4)
learning_rate = 1e-2

epochs = 5



textual_labels = ["rest", "task_motor", "task_story_math", "task_working_memory"]
label_encoder = LabelBinarizer()
label_encoder.fit(textual_labels)

model = JustLSTM((timeframe, 248), timeframe, LSTM_units, 16, output_classes)
model_type = "lstm"

optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

model, losses, accuracies = train_dir(model, model_type, "data/Intra/train_prepro", epochs, timeframe, optimizer, label_encoder)

Starting epoch 1

Fitting on files 0-8....
Fitting on file 1 containing task: task_motor
Training loss value at step 0: 1.3885600566864014
Training loss value at step 100: 0.00021252757869660854
Training loss value at step 200: 0.00016234986833296716
Training loss value at step 300: 8.582700684200972e-05
Training loss value at step 400: 8.225102646974847e-05
Training loss value at step 500: 5.364274329622276e-05
Training loss value at step 600: 2.2291887944447808e-05
Training loss value at step 700: 1.966933996300213e-05
Training accuracy over whole file: 0.9974715709686279
---------------------------------------------------
Fitting on file 2 containing task: task_motor
Training loss value at step 0: 1.9311717551317997e-05
Training loss value at step 100: 1.728519782773219e-05
Training loss value at step 200: 4.172316494077677e-06
Training loss value at step 300: 2.264974000354414e-06
Training loss value at step 400: 2.9802276912960224e-06
Training loss value at step 500: 2.14576493817

INFO:tensorflow:Assets written to: models/trained_LSTM_e2_5_20_intra/assets


In [8]:
new_model = keras.models.load_model("models/trained_LSTM_e2_5_20_intra")
new_model.summary()

Model: "just_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm1 (LSTM)                multiple                  21520     
                                                                 
 lstm2 (LSTM)                multiple                  3280      
                                                                 
 dense (Dense)               multiple                  0         
                                                                 
 output (Dense)              multiple                  84        
                                                                 
Total params: 24884 (97.20 KB)
Trainable params: 24884 (97.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
loss_arr = [tensor.numpy() for tensor in losses]

In [10]:
print(loss_arr)

[0.22823459, 0.52954596, 0.86748374, 1.2125359, 1.5186678]


In [11]:
acc_arr = [tensor.numpy() for tensor in accuracies]

In [12]:
print(acc_arr)

[3.966419, 7.916561, 11.856733, 15.78354, 19.71281]
